In [2]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

## Group Assignment
### Team Number: XX
### Team Member Names:
### Team Strategy Chosen: __________(RISKY OR SAFE)

In [3]:
## Assign csv data to a variable
hi

col_names=['Ticker']
tick_data = pd.read_csv("Tickers.csv", names=col_names, header=None)

NameError: name 'hi' is not defined

In [2]:
tick = yf.Ticker("AAPL")
tick.info()

NameError: name 'yf' is not defined

In [ ]:
temp_list = []
tick_list = []
for x in range(len(tick_data.index)):
    tick = yf.Ticker(tick_data['Ticker'].iloc[x])
    info = tick.info
    data = pd.DataFrame(info.keys())
    data.columns = ['Value']
    if data[data.Value == 'country'].empty == False :
        if 
        temp_list.append(tick_data['Ticker'].iloc[x])
print(temp_list)
start_date = '2021-07-02'
end_date = '2021-10-22'
for x in range(len(tick_list.index)):
    tick = yf.Ticker(tick_list[x])
    tick_hist = tick.history(start=start_date, end=end_date)
    avg_volume = tick_hist['Volume'].sum(axis = 0, skipna = True)/ len(tick_hist.index)
    if avg_volume >= 10000:
        tick_list.append(tick)
print(tick_list)
    
    

In [ ]:
for x in range (len(tick_list)):
    tick = yf.Ticker(tick_data['Ticker'].iloc[x])
    

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.